In [1]:
import pandas as pd
pd.set_option('display.max_columns', None) 
pd.set_option('display.width', 1000)     

In [13]:
df = pd.read_csv('data/01.csv')
df

,id_pool,id_courier,courier_type,id_job,pool_date_start,sort_order,id_general,distance,status_str,date_status,id_general.1,total_order_weight,total_orders,first_date_order,last_date_order,date_supply,date_supply_untill,date_collected,date_delivery_start,date_delivered,id_courier.1,id_service,id_source,latitude_order,longitude_order,N,shirota_tt,dolgota_tt,gen_distance
0,34769836,94477626.0,Авто,258985879,2024-01-29 18:23:16.660,3.0,252371684.0,"1,871",job_posted,2024-01-29 17:52:08.457,252371684.0,6.478,1.0,2024-01-29 17:42:59.400,2024-01-29 17:42:59.400,2024-01-29 17:42:00.000,2024-01-29 18:42:00.000,2024-01-29 17:49:34.387,2024-01-29 17:52:30.490,2024-01-29 18:26:18.703,9.447763e+07,32.0,2.0,55.716801,37.891499,5981.0,55.724824,37.910756,"1,995"
1,34769836,94477626.0,Авто,258984196,2024-01-29 17:49:37.277,1.0,252370826.0,"1,796",job_posted,2024-01-29 17:47:45.763,252370826.0,10.609,1.0,2024-01-29 17:40:58.250,2024-01-29 17:40:58.250,2024-01-29 17:40:00.000,2024-01-29 18:40:00.000,2024-01-29 17:46:30.120,2024-01-29 17:49:18.827,2024-01-29 18:18:03.330,9.447763e+07,32.0,3.0,55.714930,37.895327,5981.0,55.724824,37.910756,"1,79"
2,33310463,94477626.0,Авто,249449879,2024-01-05 09:39:55.817,1.0,243436941.0,"1,751",job_posted,2024-01-05 09:30:45.020,243436941.0,1.443,1.0,2024-01-04 20:59:24.537,2024-01-04 20:59:24.537,2024-01-05 10:00:00.000,2024-01-05 11:00:00.000,2024-01-05 04:59:53.787,2024-01-05 09:39:16.197,2024-01-05 09:57:08.760,9.447763e+07,32.0,3.0,55.714342,37.897004,5981.0,55.724824,37.910756,"1,663"
3,33310463,94477626.0,Авто,249450220,2024-01-05 10:21:46.957,7.0,243413908.0,"0,55",job_posted,2024-01-05 09:30:53.383,243413908.0,11.206,1.0,2024-01-04 19:25:41.997,2024-01-04 19:25:41.997,2024-01-05 10:00:00.000,2024-01-05 11:00:00.000,2024-01-05 05:01:31.583,2024-01-05 09:40:01.587,2024-01-05 10:22:31.890,9.447763e+07,32.0,2.0,55.709231,37.886447,5981.0,55.724824,37.910756,"2,82"
4,33319925,94477626.0,Авто,249512972,2024-01-05 12:48:11.027,8.0,243549906.0,"1,475",job_posted,2024-01-05 11:42:25.683,243549906.0,1.235,1.0,2024-01-05 11:31:19.067,2024-01-05 11:31:19.067,2024-01-05 11:31:00.000,2024-01-05 12:31:00.000,2024-01-05 11:37:07.093,2024-01-05 11:58:34.317,2024-01-05 12:50:17.340,9.447763e+07,32.0,2.0,55.721001,37.891998,5981.0,55.724824,37.910756,"1,51"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4307162,34787250,NaN,Тип курьера неизвестен,258258634,NaN,1.0,251618198.0,"4,082",job_posted,2024-01-27 17:11:29.153,251618198.0,1.250,1.0,2024-01-27 16:06:16.840,2024-01-27 16:06:16.840,2024-01-27 16:06:00.000,2024-01-27 18:06:00.000,2024-01-27 16:08:03.003,2024-01-27 17:11:34.647,2024-01-28 09:06:09.390,1.699945e+09,1.0,3.0,55.799322,37.697728,5668.0,55.781688,37.675279,"4,12"
4307163,34787252,NaN,Тип курьера неизвестен,258733083,NaN,1.0,252107083.0,"8,99",job_reposted,2024-01-29 00:03:55.250,252107083.0,2.429,1.0,2024-01-28 22:44:34.777,2024-01-28 22:44:34.777,2024-01-29 06:00:00.000,2024-01-29 08:00:00.000,2024-01-28 22:51:52.333,2024-01-29 06:01:04.637,2024-01-29 06:45:42.340,1.699922e+09,1.0,10.0,55.697688,37.623502,4799.0,55.680996,37.692869,"7,79"
4307164,34727517,NaN,Тип курьера неизвестен,258733083,NaN,1.0,252107083.0,"8,99",job_reposted,2024-01-29 00:03:55.250,252107083.0,2.429,1.0,2024-01-28 22:44:34.777,2024-01-28 22:44:34.777,2024-01-29 06:00:00.000,2024-01-29 08:00:00.000,2024-01-28 22:51:52.333,2024-01-29 06:01:04.637,2024-01-29 06:45:42.340,1.699922e+09,1.0,10.0,55.697688,37.623502,4799.0,55.680996,37.692869,"7,79"
4307165,34667086,NaN,Тип курьера неизвестен,258258634,NaN,1.0,251618198.0,"4,082",job_posted,2024-01-27 17:11:29.153,251618198.0,1.250,1.0,2024-01-27 16:06:16.840,2024-01-27 16:06:16.840,2024-01-27 16:06:00.000,2024-01-27 18:06:00.000,2024-01-27 16:08:03.003,2024-01-27 17:11:34.647,2024-01-28 09:06:09.390,1.699945e+09,1.0,3.0,55.799322,37.697728,5668.0,55.781688,37.675279,"4,12"


До того чтобы начинать надо исправить один момент:
Есть два столбцы с ид курьера (id_courier из таблицы с пулями, id_courier.1 из Reports_data). Иногда бывает что ид есть только в id_courier.1 и наоборот иногда только в id_courier, поэтому нужно дополнить один из них со значенями которых нет. Если данные о курьере есть в id_courier.1, а не в id_courier означает, что нужно еще получить данные о курьере (courier_type) из исторических данных об этом курьере, так как из-за того что там null в id_courier, тип курьера будет неизвестен.

После этого нужно дропнуть строки где нигде нет инфо о курьере

In [17]:
df['id_courier'] = df['id_courier'].astype('Int64')  # Основной ID курьера
df['id_courier.1'] = df['id_courier.1'].astype('Int64')  # Дополнительный ID курьера

df['id_courier'] = df['id_courier'].fillna(df['id_courier.1'])

# 2. Создаем маппинг известных `courier_type`, исключая NaN и "Тип курьера неизвестен"
courier_types_map = df[df['courier_type'] != "Тип курьера неизвестен"] \
    [['id_courier', 'courier_type']] \
    .dropna().drop_duplicates().set_index('id_courier')['courier_type']

# 3. Обновляем `courier_type` **Только там, где он == "Тип курьера неизвестен"**
df.loc[df['courier_type'] == "Тип курьера неизвестен", 'courier_type'] = \
    df.loc[df['courier_type'] == "Тип курьера неизвестен", 'id_courier'].map(courier_types_map)

df.dropna(subset=['id_courier'], inplace=True)
df.drop(columns=['id_courier.1'], inplace=True)
df


,id_pool,id_courier,courier_type,id_job,pool_date_start,sort_order,id_general,distance,status_str,date_status,id_general.1,total_order_weight,total_orders,first_date_order,last_date_order,date_supply,date_supply_untill,date_collected,date_delivery_start,date_delivered,id_service,id_source,latitude_order,longitude_order,N,shirota_tt,dolgota_tt,gen_distance
0,34769836,94477626,Авто,258985879,2024-01-29 18:23:16.660,3.0,252371684.0,"1,871",job_posted,2024-01-29 17:52:08.457,252371684.0,6.478,1.0,2024-01-29 17:42:59.400,2024-01-29 17:42:59.400,2024-01-29 17:42:00.000,2024-01-29 18:42:00.000,2024-01-29 17:49:34.387,2024-01-29 17:52:30.490,2024-01-29 18:26:18.703,32.0,2.0,55.716801,37.891499,5981.0,55.724824,37.910756,"1,995"
1,34769836,94477626,Авто,258984196,2024-01-29 17:49:37.277,1.0,252370826.0,"1,796",job_posted,2024-01-29 17:47:45.763,252370826.0,10.609,1.0,2024-01-29 17:40:58.250,2024-01-29 17:40:58.250,2024-01-29 17:40:00.000,2024-01-29 18:40:00.000,2024-01-29 17:46:30.120,2024-01-29 17:49:18.827,2024-01-29 18:18:03.330,32.0,3.0,55.714930,37.895327,5981.0,55.724824,37.910756,"1,79"
2,33310463,94477626,Авто,249449879,2024-01-05 09:39:55.817,1.0,243436941.0,"1,751",job_posted,2024-01-05 09:30:45.020,243436941.0,1.443,1.0,2024-01-04 20:59:24.537,2024-01-04 20:59:24.537,2024-01-05 10:00:00.000,2024-01-05 11:00:00.000,2024-01-05 04:59:53.787,2024-01-05 09:39:16.197,2024-01-05 09:57:08.760,32.0,3.0,55.714342,37.897004,5981.0,55.724824,37.910756,"1,663"
3,33310463,94477626,Авто,249450220,2024-01-05 10:21:46.957,7.0,243413908.0,"0,55",job_posted,2024-01-05 09:30:53.383,243413908.0,11.206,1.0,2024-01-04 19:25:41.997,2024-01-04 19:25:41.997,2024-01-05 10:00:00.000,2024-01-05 11:00:00.000,2024-01-05 05:01:31.583,2024-01-05 09:40:01.587,2024-01-05 10:22:31.890,32.0,2.0,55.709231,37.886447,5981.0,55.724824,37.910756,"2,82"
4,33319925,94477626,Авто,249512972,2024-01-05 12:48:11.027,8.0,243549906.0,"1,475",job_posted,2024-01-05 11:42:25.683,243549906.0,1.235,1.0,2024-01-05 11:31:19.067,2024-01-05 11:31:19.067,2024-01-05 11:31:00.000,2024-01-05 12:31:00.000,2024-01-05 11:37:07.093,2024-01-05 11:58:34.317,2024-01-05 12:50:17.340,32.0,2.0,55.721001,37.891998,5981.0,55.724824,37.910756,"1,51"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4307162,34787250,1699944619,Вело Свой,258258634,NaN,1.0,251618198.0,"4,082",job_posted,2024-01-27 17:11:29.153,251618198.0,1.250,1.0,2024-01-27 16:06:16.840,2024-01-27 16:06:16.840,2024-01-27 16:06:00.000,2024-01-27 18:06:00.000,2024-01-27 16:08:03.003,2024-01-27 17:11:34.647,2024-01-28 09:06:09.390,1.0,3.0,55.799322,37.697728,5668.0,55.781688,37.675279,"4,12"
4307163,34787252,1699922287,Авто,258733083,NaN,1.0,252107083.0,"8,99",job_reposted,2024-01-29 00:03:55.250,252107083.0,2.429,1.0,2024-01-28 22:44:34.777,2024-01-28 22:44:34.777,2024-01-29 06:00:00.000,2024-01-29 08:00:00.000,2024-01-28 22:51:52.333,2024-01-29 06:01:04.637,2024-01-29 06:45:42.340,1.0,10.0,55.697688,37.623502,4799.0,55.680996,37.692869,"7,79"
4307164,34727517,1699922287,Авто,258733083,NaN,1.0,252107083.0,"8,99",job_reposted,2024-01-29 00:03:55.250,252107083.0,2.429,1.0,2024-01-28 22:44:34.777,2024-01-28 22:44:34.777,2024-01-29 06:00:00.000,2024-01-29 08:00:00.000,2024-01-28 22:51:52.333,2024-01-29 06:01:04.637,2024-01-29 06:45:42.340,1.0,10.0,55.697688,37.623502,4799.0,55.680996,37.692869,"7,79"
4307165,34667086,1699944619,Вело Свой,258258634,NaN,1.0,251618198.0,"4,082",job_posted,2024-01-27 17:11:29.153,251618198.0,1.250,1.0,2024-01-27 16:06:16.840,2024-01-27 16:06:16.840,2024-01-27 16:06:00.000,2024-01-27 18:06:00.000,2024-01-27 16:08:03.003,2024-01-27 17:11:34.647,2024-01-28 09:06:09.390,1.0,3.0,55.799322,37.697728,5668.0,55.781688,37.675279,"4,12"


#### проверить

1. Часто ли бывает, что курьер работает на нескольких ТТ и берет с 1 ТТ - оч мало заказов ?

2. Убедиться что gen_distance показывает расстояние от тт до заказа

In [18]:
df['date_delivered'] = pd.to_datetime(df['date_delivered'])
df['date'] = df['date_delivered'].dt.date

# Проверяем, работал ли курьер на нескольких ТТ в один день
# Группируем по курьеру (`id_courier`) и дню (`date`), затем считаем количество уникальных ТТ (`id_tt`)
multiple_tt_per_day = (
    df.groupby(['id_courier', 'date'])['N']
    .nunique()
    .reset_index()
    .rename(columns={'N': 'unique_tt_count'})
)

# Фильтруем только те случаи, где количество ТТ больше 1
result = multiple_tt_per_day[multiple_tt_per_day['unique_tt_count'] > 1]
result

,id_courier,date,unique_tt_count
3,94477626,2024-01-05,2
4,94477626,2024-01-06,2
8,94477626,2024-01-12,2
19,154799142,2024-01-06,2
20,154799142,2024-01-07,2
...,...,...,...
112889,1799928811,2024-01-31,2
112904,1799928914,2024-01-30,4
112905,1799928914,2024-01-31,3
112923,1799929008,2024-01-31,2


In [12]:
df[(df['id_courier'] == 1799928914) & (pd.to_datetime(df['date']) == pd.to_datetime('2024-01-30'))]

,id_pool,id_courier,courier_type,id_job,pool_date_start,sort_order,id_general,distance,status_str,date_status,id_general.1,total_order_weight,total_orders,first_date_order,last_date_order,date_supply,date_supply_untill,date_collected,date_delivery_start,date_delivered,id_courier.1,id_service,id_source,latitude_order,longitude_order,N,shirota_tt,dolgota_tt,gen_distance,date
4306164,34815299,1799928914,Авто,259242886,2024-01-30 15:01:56.150,4.0,252549754.0,"3,839",job_posted,2024-01-30 14:00:36.397,252549754.0,2.133,1.0,2024-01-30 10:37:22.813,2024-01-30 10:37:22.813,2024-01-30 15:00:00.000,2024-01-30 16:00:00.000,2024-01-30 10:39:49.097,2024-01-30 14:12:48.737,2024-01-30 15:10:31.093,1.799929e+09,1.0,4.0,55.989300,37.228500,6850.0,56.006497,37.197565,"3,84",2024-01-30
4306165,34822579,1799928914,Авто,259281331,2024-01-30 16:10:55.187,1.0,252674676.0,"2,463",job_posted,2024-01-30 15:50:31.603,252674676.0,1.805,1.0,2024-01-30 15:42:09.763,2024-01-30 15:42:09.763,2024-01-30 15:42:00.000,2024-01-30 17:42:00.000,2024-01-30 15:50:30.830,2024-01-30 16:10:31.417,2024-01-30 16:24:57.317,1.799929e+09,1.0,4.0,55.985100,37.179699,1132.0,55.979918,37.176203,"1,004",2024-01-30
4306166,34822579,1799928914,Авто,259280474,2024-01-30 16:26:27.310,2.0,252671822.0,"1,732",job_posted,2024-01-30 15:47:41.637,252671822.0,1.738,1.0,2024-01-30 15:34:17.680,2024-01-30 15:34:17.680,2024-01-30 15:34:00.000,2024-01-30 17:34:00.000,2024-01-30 15:47:40.033,2024-01-30 16:10:07.480,2024-01-30 16:33:43.233,1.799929e+09,1.0,3.0,55.978919,37.185108,1132.0,55.979918,37.176203,"1,78",2024-01-30
4306167,34821518,1799928914,Авто,259275064,2024-01-30 15:49:36.363,1.0,252668831.0,"1,816",job_posted,2024-01-30 15:31:04.527,252668831.0,4.898,1.0,2024-01-30 15:26:03.070,2024-01-30 15:26:03.070,2024-01-30 15:26:00.000,2024-01-30 17:26:00.000,2024-01-30 15:31:03.337,2024-01-30 15:47:33.860,2024-01-30 16:09:03.683,1.799929e+09,1.0,2.0,55.978674,37.179446,4852.0,55.982688,37.174990,"1,79",2024-01-30
4306168,34819933,1799928914,Авто,259272401,2024-01-30 15:28:28.447,1.0,252665202.0,"1,122",job_posted,2024-01-30 15:24:27.560,252665202.0,4.358,1.0,2024-01-30 15:16:09.173,2024-01-30 15:16:09.173,2024-01-30 15:16:00.000,2024-01-30 17:16:00.000,2024-01-30 15:24:20.503,2024-01-30 15:28:01.803,2024-01-30 15:45:10.147,1.799929e+09,1.0,3.0,55.993082,37.218855,4825.0,55.993656,37.212980,"0,609",2024-01-30
4306169,34815299,1799928914,Авто,259249281,2024-01-30 14:48:22.237,3.0,252640635.0,"4,172",job_posted,2024-01-30 14:19:50.067,252640635.0,15.307,1.0,2024-01-30 14:10:11.490,2024-01-30 14:10:11.490,2024-01-30 14:10:00.000,2024-01-30 16:10:00.000,2024-01-30 14:17:40.427,2024-01-30 14:20:38.040,2024-01-30 15:01:56.157,1.799929e+09,1.0,2.0,55.989470,37.232186,6850.0,56.006497,37.197565,"4,18",2024-01-30
4306170,34815299,1799928914,Авто,259248804,2024-01-30 14:37:14.187,2.0,252638865.0,"4,097",job_posted,2024-01-30 14:18:06.443,252638865.0,2.590,1.0,2024-01-30 14:03:15.327,2024-01-30 14:03:15.327,2024-01-30 14:03:00.000,2024-01-30 16:03:00.000,2024-01-30 14:14:37.447,2024-01-30 14:18:35.993,2024-01-30 14:48:22.247,1.799929e+09,1.0,3.0,55.988731,37.229320,6850.0,56.006497,37.197565,"4,1",2024-01-30
4306171,34815299,1799928914,Авто,259245581,2024-01-30 14:13:33.337,1.0,252636535.0,"4,342",job_posted,2024-01-30 14:07:29.263,252636535.0,3.960,1.0,2024-01-30 13:57:24.733,2024-01-30 13:57:24.733,2024-01-30 13:57:00.000,2024-01-30 15:57:00.000,2024-01-30 14:04:25.477,2024-01-30 14:12:37.803,2024-01-30 14:37:14.197,1.799929e+09,1.0,3.0,55.998864,37.237188,6850.0,56.006497,37.197565,"4,49",2024-01-30


2. Проверка gen_distance

In [30]:
import osmnx as ox
import networkx as nx
from geopy.distance import geodesic

i = 0

# Ensure latitude and longitude columns are named correctly
lat1, lon1 = df.iloc[i]['latitude_order'], df.iloc[i]['longitude_order']  # First coordinate
lat2, lon2 = df.iloc[i]['shirota_tt'], df.iloc[i]['dolgota_tt']  # Second coordinate
gen_dist = df.iloc[i]['gen_distance']

# Calculate the geodesic distance (in kilometers)
distance_km = geodesic((lat1, lon1), (lat2, lon2)).kilometers

# Print the result
print(f"Distance for the {i} row: {distance_km:.2f} km, {gen_dist}")

# Download the street network from OpenStreetMap
G = ox.graph_from_point((lat1, lon1), dist=2000, network_type="drive") 

# Get the nearest nodes in the graph to our coordinates
orig_node = ox.distance.nearest_nodes(G, lon1, lat1)
dest_node = ox.distance.nearest_nodes(G, lon2, lat2)

# Compute the shortest path distance using real streets
distance_meters = nx.shortest_path_length(G, orig_node, dest_node, weight="length")
distance_km = distance_meters / 1000  # Convert to kilometers

# Print the street-based distance
print(f"Street distance for the {i} row: {distance_km:.2f} km")



Distance for the 0 row: 1.50 km, 1,995
Street distance for the 0 row: 1.42 km


### Обработка по курьерам

В конце нужно будет иметь таблицу со следующими данными:
* *ид курьера, тт, дата время, статус, колво заказов на весь день в тот ТТ, колво пулов на весь день в тот ТТ, сколько часов работал на тот день, 

Статистики, которые нужно будет отдельно считать и потом джоинить с таблицей выше:
1. Среднее кол-во заказов в час 
2. Среднее кол-во пулов в час
3. Ср кол-во часов в смене в будни 
4. Ср кол-во часов в смене в выходные 
5. Кол-во смен в день ( утром + вечер и тд)
6. Ср вес заказа
7. Ср. время доставки 1 пула
8. Ср. время доставки 1 заказа
9. Ср. время между взятие следующего пула или заказа и delivered
10. Ср. скорость доставки 
11. ср. расстояние до заказа (gen_distance)

Также нужно удалить строки где date_status до date_order - это аномалия, которая означает, что заказ появился в приложение курьера до того когда этот заказ заказал клиент


In [ ]:
# код


Начнем с данными на тот день работы:

Добавим дату в формате datetime (<b> * date_group </b>) для каждой строки на основе date_delivery_start (когда курьер выбрал заказ), если дата 01.02.2024 13:24:22 значит дата и время будет 01.02.2024 13:15, т.е 
hh:00-hh:15 - пишется как hh:00
hh:15-hh:30 - пишется как hh:15
hh:30-hh:45 - пишется как hh:30
hh:45-hh:00 - пишется как hh:45



По группе id_courier, tt, date_group считать <b> * колво выбранных заказов в тот день </b> курьерам (date_delivery_start)


Для того чтобы определить колво пулов нужно будет добавить тоже дату начала и конца пула.
- чтобы понять время начала пула можно смотреть на date_delivered первого заказа в пуле и на основе gen_distance вычесть несколько минут чтобы учитывать когда он из ТТ уехал (date_start_pool)
- чтобы понять время конца пула смотреть на date_delivered последнего заказа и добавить несколько минут также на основе gen_distance (date_finish_pool)




Считать время возврата курьера в ТТ.
Для этого можно использовать date_delivered последнего заказа, gen_distance, или самим попробовать считать distance между координатами чтобы убедиться что gen_distance реально показывает дистанцию между заказом и тт.

Зачем считать? Мы можем потом найти среднее время возврата и на основе этого понять когда он заканчивал смену 

На основе время начала и конца добавить столбец <b>* статуса</b> курьера:
- в пути (если date_group между date_start_pool)
- в тт (если есть date_delivery_start какого-то заказа и date_finish_pool до date_delivery_start)
- дома 🐣 (если за больше чем среднее время возврата в ТТ он не выбрал заказы)

Также добавить для каждого date_group взял ли пул курьера в это время <b>(deliver_pool)</b>? 1 - если да, 0 - если нет 

Потом считать <b>* колво пулов</b> по группе id_courier, tt, date_group (просто sum deliver_pool)

Для того чтобы считать <b> * сколько он работал тот день </b> можем считать как:

сумма(date_delivered последнего заказа в пуле - date_delivery_start первого заказа в пуле)


С другой стороны, необходимо будет рассчитать историческую статистику, которую мы объединим с данными, рассчитанными выше
1. Среднее кол-во заказов в час 
2. Среднее кол-во пулов в час
3. Ср кол-во часов в смене в будни 
4. Ср кол-во часов в смене в выходные 
5. Кол-во смен в день ( утром + вечер и тд)
6. Ср вес заказа
7. Ср. время доставки 1 пула
8. Ср. время доставки 1 заказа
9. Ср. время между взятие следующего пула или заказа и delivered
10. ср. расстояние до заказа (gen_distance)

Тут статистики по часам так что создаем опять date_hour_group и все стольцы как описали выше только тут по часам и после этого возьмем среднее:
1. Среднее кол-во заказов в час 
2. Среднее кол-во пулов в час

1. Для расчета остальной статистики необходимо будет создать дату по дням (date_day) на основе date_delivery_start и date_delivered (чтобы точно определить какой этот день например если date_delivery_start в 23:59, а date_delivered в 01:00, пул относится больше к второму дню)

2. После этого считаем колво часов сколько чел работал на основе доставленных пулов. Для этого можем использовать сумма(date_delivered последнего заказа в пуле - date_delivery_start первого заказа в пуле).

3. Считаем колво смен. Для этого смотрим разницу по времени между date_delivered первой пули и date_delivery_start второй пули. Либо сравниваем его с среднем, либо просто поставим час или часа 2 и больше этого значит две смены.

4. Посчитаем сколько часов работал утром, днем, вечером, ночью 


5. Добавим столбец выходной ли день

6. Чтобы считать среднее колво часов в смене в будни или выходные:
    a. используем данные которые считали выше: 
        колво часов = сколько часов работал/ колво смен 
    b. группируем по ид, день, выходной ли день. 

7. Можем считать среднее колво часов по частям дням: группируем по ид, день, агрегация: среднее по вечер, день, утро, ночь



Чтобы считать остальные статистики:
Ср вес заказа и Ср. время доставки 1 пула, Ср. время доставки 1 заказа, ср. расстояние до заказа (gen_distance):

    a. группируем по id_pool id_courier, и агрегация: sum (order_weight), last(date_delivered), last(date_delivery_start), count(id_general) - колво заказов в пуле, mean(gen_distance)

    b. считаем разницу pool_time = last(date_delivered) - last(date_delivery_start)

    с. группируем по id_courier, и агрегация: mean (order_weight), mean(pool_time), mean(count(id_general)), mean(mean(gen_distance))

Для Ср. время между пулами:

    a. группируем по id_courier, id_pool, агрегация: last(date_delivered), first(date_delivered), last(date_delivery_start), first(date_delivery_start)

    b. посчитаем второй пул(first(date_delivered) -(first(date_delivered) - last(date_delivery_start))) - первый пул(last(date_delivered) + (last(date_delivered) - first(date_delivery_start)))

(date_delivered первого заказа - нескок мин от тт до заказа) - понять когда начиналась доставка пула. Можем предлагать что поледний момент когда чел был в тт был когда добавил последний заказ в пул, last(date_delivery_start)

(date_delivered последнего заказа + нескок мин от заказа до тт) - понять когда заканчивалась доставка и чел вернулся в тт. Тут можем предлагать что при добавлении заказа для следующего пула, курьер в тт

### Обработка по ТТ